In [2]:
import pandas as pd
import numpy as np
import math
import time

import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import torch.nn.functional as F

In [3]:
start=time.time()
data = np.load("data_preprocessed.npy",allow_pickle=True)
data=pd.DataFrame(data)
cols = list(np.load("data_columns.npy",allow_pickle=True))
data.columns=cols
print(time.time()-start)

134.71592783927917


In [6]:
copie=data.copy()

## Put the data in the right formats

In [267]:
df_sup = copie.loc[copie["retweet_count"]>1000]
df_below_all = copie.loc[copie["retweet_count"]<=1000].sample(frac = 1).reset_index(drop=True)
df_below = df_below_all[:100000]
data_balanced = pd.concat([df_sup,df_below])

In [268]:
data_balanced = data_balanced.sample(frac = 1) 
data_balanced = data_balanced.reset_index(drop=True)

In [269]:
def create_data(data) :
    count = 0
    nb = 50000
    bad =0
    X_all = np.zeros((data.shape[0],38))
    y_all = np.zeros((data.shape[0],1))
    X = np.zeros((data.shape[0],38))
    y = np.zeros((data.shape[0],1))
    for k in range(data.shape[0]) :
        current = data.iloc[k]
        if current.retweet_count == 0 and count < nb :
            y[k,0] = current.retweet_count
            X[k,0] = current.user_verified
            X[k,1] = current.user_statuses_count
            X[k,2] = current.user_followers_count
            X[k,3] = current.user_friends_count
            X[k,4] = current.user_mentions
            X[k,5] = current.urls
            X[k,6] = current.hashtags
            X[k,7:14] = current.day
            X[k,14:38] = current.time
            count+=1
        elif current.retweet_count != 0 :
            y[k,0] = current.retweet_count
            X[k,0] = current.user_verified
            X[k,1] = current.user_statuses_count
            X[k,2] = current.user_followers_count
            X[k,3] = current.user_friends_count
            X[k,4] = current.user_mentions
            X[k,5] = current.urls
            X[k,6] = current.hashtags
            X[k,7:14] = current.day
            X[k,14:38] = current.time
        else : 
            bad +=1
        
        
        y_all[k,0] = current.retweet_count
        X_all[k,0] = current.user_verified
        X_all[k,1] = current.user_statuses_count
        X_all[k,2] = current.user_followers_count
        X_all[k,3] = current.user_friends_count
        X_all[k,4] = current.user_mentions
        X_all[k,5] = current.urls
        X_all[k,6] = current.hashtags
        X_all[k,7:14] = current.day
        X_all[k,14:38] = current.time
    X = X[:data.shape[0]-bad,:]
    y = y[:data.shape[0]-bad]
    return X,y,X_all,y_all

In [270]:
#for classification
def create_data_classif(data) :
    bad =0
    X = np.zeros((data.shape[0],38))
    y = np.zeros((data.shape[0],1))
    for k in range(data.shape[0]) :
        current = data.iloc[k]
        if current.retweet_count < 1000:
            y[k,0] = 0
            X[k,0] = current.user_verified
            X[k,1] = current.user_statuses_count
            X[k,2] = current.user_followers_count
            X[k,3] = current.user_friends_count
            X[k,4] = current.user_mentions
            X[k,5] = current.urls
            X[k,6] = current.hashtags
            X[k,7:14] = current.day
            X[k,14:38] = current.time
            
        elif current.retweet_count >= 1000 :
            y[k,0] = 1
            X[k,0] = current.user_verified
            X[k,1] = current.user_statuses_count
            X[k,2] = current.user_followers_count
            X[k,3] = current.user_friends_count
            X[k,4] = current.user_mentions
            X[k,5] = current.urls
            X[k,6] = current.hashtags
            X[k,7:14] = current.day
            X[k,14:38] = current.time
            
        else : 
            bad +=1
            
    X = X[:data.shape[0]-bad,:]
    y = y[:data.shape[0]-bad]
    return X,y

In [271]:
X_sup, y_sup, X_all_sup, y_all_sup = create_data(df_sup)
X_below, y_below, X_all_below, y_all_below = create_data(df_below_all)
X_classif, y_classif = create_data_classif(data_balanced)

In [272]:
X_sup = torch.from_numpy(X_sup).float()
y_sup = torch.from_numpy(y_sup).float()
X_all_sup = torch.from_numpy(X_all_sup).float()
y_all_sup = torch.from_numpy(y_all_sup).float()

X_below = torch.from_numpy(X_below).float()
y_below = torch.from_numpy(y_below).float()
X_all_below = torch.from_numpy(X_all_below).float()
y_all_below = torch.from_numpy(y_all_below).float()

## Training of the classification model

In [173]:
def create_batch(X,y,size=64) :
    res = []
    for k in range(X.shape[0]//size) :
        try:
            res.append([X[k*size:(k+1)*size,:],y[k*size:(k+1)*size]])
        except :
            res.append([X[k*size:,:],y[k*size]])
    return res

In [175]:
def train(model, device, train_loader, optimizer, epoch):
    log_interval=400
    losss = nn.L1Loss()
    model.train() #set model in train mode
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = losss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader)*data.shape[0],
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval() #set model in test mode
    test_loss = 0
    correct = 0
    losss = nn.L1Loss()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += losss(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader),
        100. * correct / len(test_loader.dataset)))

In [176]:
device = torch.device("cpu") #if you have a GPU with CUDA installed, this may speed up computation

In [273]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=30, random_state=0)
clf.fit(X_classif, y_classif)

C:\Users\alexi\anaconda3\envs\inf573\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(max_depth=30, random_state=0)

In [274]:
pred = clf.predict(X_classif)
prec=len(y_classif)
count=0
for i in range(len(y_classif)):
    if y_classif[i]!=pred[i]:
        prec-=1
    elif pred[i]==1:
        count+=1
print("precision : "+str(prec/len(y_classif)*100)+" %")
print(count)

precision : 99.4239840348331 %
9632


In [275]:
pred = clf.predict(X_all_below)
prec=len(y_all_below)
for i in range(len(y_all_below)):
    if pred[i]==1.:
        prec-=1
print("precision : "+str(prec/len(y_all_below)*100)+" %")

precision : 98.36637748898994 %


In [276]:
pred = clf.predict(X_all_sup)
prec=len(y_all_sup)
for i in range(len(y_all_sup)):
    if pred[i]==0.:
        prec-=1
print("precision : "+str(prec/len(y_all_sup)*100)+" %")

precision : 94.052734375 %


## Training for the low model

In [186]:
class low_model(nn.Module):
    def __init__(self,input_size):
        super(low_model, self).__init__()
        #insert your code here
        self.linear1 = nn.Linear(input_size,128)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(128,1)
        
        
    def forward(self, x):
        #initial dimensions for x will be [64, 1, 28, 28]
        #insert your code here
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        
        return x

In [187]:
model_low = low_model(38)

In [188]:
optimizer = torch.optim.RMSprop(model_low.parameters(), lr=0.01) #actually SGD is just GD in this case

In [189]:
train_loader = create_batch(X_below[:240000,:],y_below[:240000,:],size=64)
test_loader = create_batch(X_below[240000:,:],y_below[240000:],size=64)

In [190]:
for epoch in range(1,10) :
    train(model_low, device, train_loader, optimizer, epoch)    

Train Epoch: 1 [0/240000 (0%)]	Loss: 33.778866
Train Epoch: 1 [25600/240000 (11%)]	Loss: 11.941899
Train Epoch: 1 [51200/240000 (21%)]	Loss: 18.180136
Train Epoch: 1 [76800/240000 (32%)]	Loss: 8.201487
Train Epoch: 1 [102400/240000 (43%)]	Loss: 28.475744
Train Epoch: 1 [128000/240000 (53%)]	Loss: 4.115740
Train Epoch: 1 [153600/240000 (64%)]	Loss: 14.874025
Train Epoch: 1 [179200/240000 (75%)]	Loss: 14.555970
Train Epoch: 1 [204800/240000 (85%)]	Loss: 17.904186
Train Epoch: 1 [230400/240000 (96%)]	Loss: 14.273594
Train Epoch: 2 [0/240000 (0%)]	Loss: 34.351364
Train Epoch: 2 [25600/240000 (11%)]	Loss: 11.936251
Train Epoch: 2 [51200/240000 (21%)]	Loss: 18.245010
Train Epoch: 2 [76800/240000 (32%)]	Loss: 7.973222
Train Epoch: 2 [102400/240000 (43%)]	Loss: 28.484110
Train Epoch: 2 [128000/240000 (53%)]	Loss: 4.296146
Train Epoch: 2 [153600/240000 (64%)]	Loss: 14.789101
Train Epoch: 2 [179200/240000 (75%)]	Loss: 14.213564
Train Epoch: 2 [204800/240000 (85%)]	Loss: 18.047682
Train Epoch: 2 

## Training for the sup model

In [195]:
class sup_model(nn.Module):
    def __init__(self,input_size):
        super(sup_model, self).__init__()
        #insert your code here
        self.linear1 = nn.Linear(input_size,128)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(128,1)
        
        
    def forward(self, x):
        #initial dimensions for x will be [64, 1, 28, 28]
        #insert your code here
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
         
        return x

In [196]:
model_sup = sup_model(38)

In [198]:
optimizer = torch.optim.RMSprop(model_sup.parameters(), lr=0.01) #actually SGD is just GD in this case

In [199]:
train_loader = create_batch(X_sup[:9000,:],y_sup[:9000,:],size=64)
test_loader = create_batch(X_sup[9000:,:],y_sup[9000:],size=64)

In [200]:
for epoch in range(1,10) :
    train(model_sup, device, train_loader, optimizer, epoch) 

Train Epoch: 1 [0/8960 (0%)]	Loss: 5236.511230
Train Epoch: 2 [0/8960 (0%)]	Loss: 3711.461182
Train Epoch: 3 [0/8960 (0%)]	Loss: 3668.201172
Train Epoch: 4 [0/8960 (0%)]	Loss: 3664.354248
Train Epoch: 5 [0/8960 (0%)]	Loss: 3667.925293
Train Epoch: 6 [0/8960 (0%)]	Loss: 3673.138672
Train Epoch: 7 [0/8960 (0%)]	Loss: 3681.455322
Train Epoch: 8 [0/8960 (0%)]	Loss: 3684.517090
Train Epoch: 9 [0/8960 (0%)]	Loss: 3688.407227


In [207]:
import random
for i in range(0,20):
    n = random.randint(0,y_sup[9000:].shape[0])
    print(n," : ",int(y_sup[9000:][n]),int(model_sup(X_sup[9000:,:][n])))

1192  :  6102 3134
748  :  1950 3092
1089  :  1132 2599
586  :  1657 3590
25  :  3476 2942
343  :  3553 3326
155  :  1998 2373
1215  :  1066 2669
726  :  2015 2961
863  :  5099 2743
469  :  22877 3007
379  :  2917 2362
785  :  9952 2974
1164  :  21648 2859
559  :  1231 2806
182  :  3336 3973
861  :  1151 2799
298  :  3016 3670
588  :  2724 3719
980  :  1156 2918


## Test

In [215]:
evaluation = np.load("test_preprocessed.npy",allow_pickle=True)
evaluation=pd.DataFrame(evaluation)
cols = list(np.load("test_columns.npy",allow_pickle=True))
evaluation.columns=cols

In [216]:
evaluation

,id,user_verified,user_statuses_count,user_followers_count,user_friends_count,user_mentions,urls,hashtags,day,time
0,665776,0,-0.420814,-0.0953838,-0.153247,0,0,0,"[0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,665777,1,0.663227,-0.0784991,-0.14009,0,0,0,"[0, 0, 0, 0, 0, 0, 1]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,665778,0,-0.178158,-0.0952573,-0.147393,0,0,0,"[1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,665779,1,-0.415311,-0.0927146,-0.154174,0,1,0,"[0, 1, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,665780,0,-0.0921758,-0.0927412,-0.0890879,0,1,0,"[0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...
285329,951105,1,-0.316199,2.56861,-0.152087,0,1,3,"[0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
285330,951106,0,-0.421606,-0.0953556,-0.153189,0,1,0,"[0, 0, 0, 0, 1, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
285331,951107,0,-0.419271,-0.095367,-0.146813,0,0,0,"[1, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
285332,951108,1,-0.0780925,0.303514,-0.139279,0,0,0,"[0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [217]:
def create_data_test(data) :
    X_all = np.zeros((data.shape[0],38))
    y_all = np.zeros((data.shape[0],1))
    
    for k in range(data.shape[0]) :
        current = data.iloc[k]
        y_all[k,0] = current.id
        X_all[k,0] = current.user_verified
        X_all[k,1] = current.user_statuses_count
        X_all[k,2] = current.user_followers_count
        X_all[k,3] = current.user_friends_count
        X_all[k,4] = current.user_mentions
        X_all[k,5] = current.urls
        X_all[k,6] = current.hashtags
        X_all[k,7:14] = current.day
        X_all[k,14:38] = current.time

    return X_all,y_all

In [221]:
X_test,y_test = create_data_test(evaluation)

In [222]:
X_test = torch.from_numpy(X_test).float()

In [280]:
pred = clf.predict(X_test)
res=[]
for i in range(len(pred)):
    if pred[i]:
        res.append(int(model_sup(X_test[i])))
    else:
        res.append(int(model_low(X_test[i])))

In [281]:
df=pd.DataFrame()
df["TweetID"]=list(int(y) for y in y_test)
df["NoRetweets"]=res
df.to_csv("res2.csv",index=False,sep=",")

In [277]:
pred = clf.predict(X_all_below)
err=0
for i in range(len(pred)):
    if pred[i]:
        err+=abs(int(model_sup(X_all_below[i]))-y_all_below[i])
    else:
        err+=abs((int(model_low(X_all_below[i])))-y_all_below[i])
print(err/len(copie))
        
pred = clf.predict(X_all_sup)
err2=0
for i in range(len(pred)):
    if pred[i]:
        err2+=abs(int(model_sup(X_all_sup[i]))-y_all_sup[i])
    else:
        err2+=abs((int(model_low(X_all_sup[i])))-y_all_sup[i])
print(err2/len(copie))

print((err+err2)/len(copie))

tensor([58.6312])
tensor([105.0264])
tensor([163.6576])
